In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import astropy.wcs
from astropy.io import fits
import astropy.io
from IPython.display import clear_output
from astropy.stats import sigma_clipped_stats
from photutils import datasets
from photutils import DAOStarFinder
from photutils import find_peaks
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import CircularAperture
from photutils import RectangularAperture
from matplotlib.backends.backend_pdf import PdfPages

df = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e8.csv")
#df = pd.read_csv("test8.csv")
angle = df["Angle"].values
x1 = df["x_1"].values
x2 = df["x_2"].values
X = df["X"].values
Y = df["Y"].values
rectangle_wide = (x2 - x1)/2
filename_xml = "d_xml_ucac/sub" + df["ID"] + "_ucac.xml"
filename_fits = "d_fits/sub" + df["ID"] + ".fits"
output_filename = "sub" + df["ID"] + "_star.csv"
with open("d_xml_ucac/sub571226_095_ucac.xml") as f:
    a = f.readlines()
    b = a[92]#<TABLEDATA>
    c = a[-5] #</TABLEDATA>

############################################################################
for i in range(len(filename_xml)):
#for i in range(3):
    print filename_xml[i],i
    xml_all = np.arange(8).reshape(8)
    xml_list = np.arange(4).reshape(4)
    pixel_sextractor = np.arange(2).reshape(2)
    
###################################### xml ファイルの読み込み #######
    with open(filename_xml[i]) as f:
        l = f.readlines()
        for j in range(len(l)):
            m = l[j]
            if m == b:
                del_start = j + 1
            elif m == c:
                del_end = j
        l = l[del_start:del_end]
        xml = l[0]
        xml = re.split('<TR><TD>|</TD><TD>|</TD></TR>', xml)
        del xml[0],xml[3:14],xml[4:]
        for k in l:
            s = re.split('<TR><TD>|</TD><TD>|</TD></TR>', k)
            del s[0],s[3:14],s[4:]
            if s != ['', '', '', '','']:
                xml_list = np.vstack((xml_list, s))
    
    xml_list = np.delete(xml_list,0, 0)
    source_ID = xml_list[:,2]
    source_ID = source_ID.reshape(len(source_ID),1)
    #xml_list = np.delete(xml_list,2,axis=1)
    
### 今はsource_IDとxml_listをhstackさせちゃうけど　sextractor でどうせ
##まだできなくなるから,source_IDを別にしておく
    columns = ["Ra_catalog", "Dec_catalog", "source_ID","VMag"]
    df = pd.DataFrame(data = xml_list, columns = columns)
    df.to_csv(output_filename[i])
clear_output()

print "Complete!"


Complete!


In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import astropy.wcs
from astropy.io import fits
import astropy.io
from IPython.display import clear_output
from astropy.stats import sigma_clipped_stats
from photutils import datasets
from photutils import DAOStarFinder
from photutils import find_peaks
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import CircularAperture
from photutils import RectangularAperture
from matplotlib.backends.backend_pdf import PdfPages

df = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e8.csv")
#df = pd.read_csv("test8.csv")
filename_csv = "d_csv_ucac/sub" + df["ID"] + "_star.csv"
filename_fits = "d_fits/sub" + df["ID"] + ".fits"
output_filename = "sub" + df["ID"] + "_star.csv"
with open("d_xml_ucac/sub571226_095_ucac.xml") as f:
    a = f.readlines()
    b = a[92]#<TABLEDATA>
    c = a[-5] #</TABLEDATA>

############################################################################
for i in range(64,len(filename_xml)):
#for i in range(3):
    print filename_xml[i],i
    xml_all = np.arange(8).reshape(8)
    #xml_list = np.arange(4).reshape(4)
    pixel_sextractor = np.arange(2).reshape(2)
    fits = astropy.io.fits.open(filename_fits[i])
    hdul = fits
    data = hdul[0].data
    data = data[1,:,:]#差分前のフレームだけにする
    data = np.reshape(data, (282, 500))#2次元化
    wcs = astropy.wcs.WCS(fits[0].header, naxis = 2)
    
############## Source detector　か Peak Detector #######
    
    mean, median, std = sigma_clipped_stats(data, sigma=3.0)    
    daofind = DAOStarFinder(fwhm=3.0, threshold=50.*std)    ##############ここの設定をみる必要あり
    sources = daofind(data - median)                        ### souurce detection じゃなくて peak detection
    for col in sources.colnames:                            ###のほうがいいかもしれない
        sources[col].info.format = '%.8g'
    positions = (sources['xcentroid'], sources['ycentroid'])
    
###################################### xml ファイルの読み込み #######
    df_ucac = pd.read_csv(filename_csv[i])
    source_ID = df_ucac["source_ID"].values
    v_mag_star = df_ucac["VMag"].values
    ra_catalog = df_ucac["Ra_catalog"].values
    dec_catalog = df_ucac["Dec_catalog"].values
    pixel_coordinates = np.arange(2).reshape(1,2)
    for k in range(len(source_ID)):
                pixel_coord = wcs.all_world2pix(ra_catalog[k], dec_catalog[k], 1)
                pixel_coordinates = np.vstack((pixel_coordinates, pixel_coord))
    pixel_coordinates = np.delete(pixel_coordinates,0, 0)
    v_mag_star = np.vstack((v_mag_star,source_ID,ra_catalog,dec_catalog))
    v_mag_star = v_mag_star.transpose()#["v_mag","ID","ra_catalog","dec_catalog","x_astropy","y_astropy"]
    v_mag_star = np.hstack((v_mag_star,pixel_coordinates))
    v_mag_star = v_mag_star[np.argsort(v_mag_star[:, 0])] #等級でソート
    v_mag_star = np.delete(v_mag_star, np.where(v_mag_star[:,0]<10.), 0)
    v_mag_star = np.delete(v_mag_star, np.where(v_mag_star[:,0]>16.), 0)
    
    #x,y座標がフレーム外のindex行をxmkから削除
    #apertureが取れるように10pix余裕とる
    index = np.where((v_mag_star[:,4]>490.)|(v_mag_star[:,4]<10.)|\
                    (v_mag_star[:,5]>272.) | (v_mag_star[:,5]<10.))
    v_mag_star = np.delete(v_mag_star,index, 0)

#####################################
# Source Extractor と比較
    pix_sextractor = np.array(positions)
    pix_sextractor = pix_sextractor.transpose()
    use_index = []
    for m in range(len(pix_sextractor)):
    #for m in range(1):
        index = abs(v_mag_star[:,4] - pix_sextractor[m,0])+\
                abs(v_mag_star[:,5] - pix_sextractor[m,1])
        if index.min() < 4.0:#候補がないやつを除く
            a = index.argmin()
            use_index = np.hstack((use_index,a))
            use_index = use_index.astype(np.int64)
            pixel_sextractor = np.vstack((pixel_sextractor,pix_sextractor[m,:]))
    
    pixel_sextractor = np.delete(pixel_sextractor,0,0)
    v_mag_star = v_mag_star[use_index,:]
    v_mag_star = np.hstack((v_mag_star,pixel_sextractor))
    xml_all = np.vstack((xml_all,v_mag_star))
    xml_all = np.delete(xml_all,0,0)
    clear_output()
    #plt.imshow(data, cmap = 'gray', vmin = 0, vmax = 400, interpolation = 'none')
    #plt.ylim(0, 282)
    #plt.scatter(xml_all[:,3],xml_all[:,4],alpha = 0.5)
    #rectangular.plot(color='yellow', lw=0.5, alpha=5.)
    #apertures = CircularAperture(positions_peak, r=5.)
    #apertures = CircularAperture(positions, r=5.)
    #apertures.plot(color='green', lw=1.5, alpha=0.5)
    #pdf = PdfPages("filename.pdf") #図をpdfに出力
    #pdf.savefig()
    #pdf.close()
    #plt.show()
    
### 今はsource_IDとxml_listをhstackさせちゃうけど　sextractor でどうせ
##まだできなくなるから,source_IDを別にしておく
    columns = ["VMag","sourceID","Ra_catalog","Dec_catalog","x_astropy","y_astropy",\
               "x_sextractor","y_sextractor"]
    df = pd.DataFrame(data = xml_all, columns = columns)
    df.to_csv(output_filename[i])
clear_output()
print "Complete!"

Complete!


In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

#csvファイルの空欄を消す
df = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e8.csv")
#df = pd.read_csv("test8.csv")
filename_csv = "d_csv_ucac_used/sub" + df["ID"] + "_star.csv"
output_filename = "sub" + df["ID"] + "_star.csv"
for i in range(64,len(filename_csv)):
#for i in range(3):
    print i
    df_ucac = pd.read_csv(filename_csv[i])
    columns = df_ucac.columns
    v_mag_star = df_ucac["VMag"].values
    df_ucac = df_ucac.values
    #print df_ucac.shape
    v_mag_star = v_mag_star.astype('str')
    NaN = "nan"
    delete_index = np.where(v_mag_star == NaN)
    #print delete_index
    df_ucac = np.delete(df_ucac,delete_index,0)
    #print df_ucac.shape
    photometry_table = pd.DataFrame(data=df_ucac,columns=columns)
    photometry_table.to_csv(output_filename[i])
    clear_output()
clear_output()
print "Complete!"

Complete!
